In [1]:
import pandas as pd
import numpy as np
from math import floor
import math
import os
from datetime import datetime
import xarray as xr
from tqdm.notebook import tqdm

# Listado de playas con avistamientos y fechas

In [2]:
avistamientos_df = pd.read_excel("./ExcelsAvistamientosIniciales/Physalia_Ambiental_R.xlsx")
# avistamientos_df = pd.read_excel("../Physala_Data/Datos_Physalia_20171010.xls")

columnas = avistamientos_df.iloc[0]

#Quito las 3 primeras filas, debido al formato de la excell
avistamientos_df = avistamientos_df.iloc[3:] 
avistamientos_df.columns = columnas

# Me quedo solo con los datos de avistamientos
avistamientos_df = avistamientos_df[["Latitud","Longitud","Año","Mes","Día","Avistamientos"]]

#Transdormaciones para sacar con fecha (datetime)
avistamientos_fecha_df=avistamientos_df[["Año","Mes","Día"]]
avistamientos_fecha_df.columns = ["year","month","day"]

fecha = pd.to_datetime(avistamientos_fecha_df)

avistamientos_df["Fecha"]=fecha
avistamientos_df=avistamientos_df[["Latitud","Longitud","Fecha","Avistamientos"]]
avistamientos_df.head()

,Latitud,Longitud,Fecha,Avistamientos
3,-25.4,-70.4833,2014-05-01,1
4,-18.4631,-70.3053,2014-05-01,1
5,-27.0658,-70.8259,2014-05-09,13
6,-18.4933,-70.3261,2014-05-10,15
7,-27.122,-70.8576,2014-05-10,23


# Exporto listado de playas sin redondear coordenadas

In [3]:
# Genera excel con las coordenadas de las playas
playas_df = avistamientos_df.reset_index()
playas_df = playas_df[['Latitud','Longitud']]
playas_df.to_excel('./1AvistamientosSinRedondeoCoord.xlsx')

In [4]:
base = 1/4
def redondeo(x,y):
    return  [base * round(x/base),base * round(y/base)]
x = redondeo(37.1707222222222,-73.2053333333333)
x

[37.25, -73.25]

# Redondeo a cada cuarto de grado
Redondeo de latitud y longitud para juntar lecturas de una misma playa con coordenadas muy proximas

Se exporta el dataframe generado a un excel

In [5]:
base=1/4
def floorQuarter(x):
    return (floor(x * 4) / 4.0) 

def redondeo(x):
    return  base * round(x/base)

#Se añaden atributos con la longitud y latitud redondeadas a cada cuarto de grado
avistamientos_df["Lat_floor"] = avistamientos_df.Latitud.map(redondeo)
avistamientos_df["Long_floor"] = avistamientos_df.Longitud.map(redondeo)

df = avistamientos_df[['Lat_floor', 'Long_floor',"Fecha","Avistamientos"]]
df = df.set_index(['Lat_floor', 'Long_floor',"Fecha"])

#DataFrame total avistamientos de playas en esa cuadricula
df_sum = df.groupby(['Lat_floor', 'Long_floor',"Fecha"]).sum()

#DataFrame número de playas en esa cuadricula
df_count = df.groupby(['Lat_floor', 'Long_floor',"Fecha"]).count()

#Dataframe con el total de avistamientos y el número de playas
df_join = df_sum.join(df_count,lsuffix="I",rsuffix="R")
df_join.columns=['Suma', 'N_Playas']
df_join['Media_Playas'] = df_join.Suma /df_join.N_Playas

# DataFrame de avistamientos procesado
df_join.to_excel("2AvistamientosConRedondeoCoord-025.xlsx")

df_join.head()

Suma  N_Playas  Media_Playas
Lat_floor Long_floor Fecha                                   
-42.75    -74.00     2015-05-27     1         1           1.0
-41.75    -73.75     2014-06-04     1         1           1.0
                     2015-06-10    20         1          20.0
                     2015-06-11   151         1         151.0
          -73.50     2015-05-23     1         1           1.0

# Redondeo con salto entre cuadrantes de dataSet original (0,0833)
Redondeo de latitud y longitud para juntar lecturas de una misma playa con coordenadas muy proximas

Se exporta el dataframe generado a un excel

In [6]:
base=1/12
def floorTwelfth(x):
    return (floor(x * 12) / 12.0)

def redondeo(x):
    return  base * round(x/base)

#Se añaden atributos con la longitud y latitud redondeadas a cada cuarto de grado
avistamientos_df["Lat_floor"] = avistamientos_df.Latitud.map(redondeo)
avistamientos_df["Long_floor"] = avistamientos_df.Longitud.map(redondeo)

df = avistamientos_df[['Lat_floor', 'Long_floor',"Fecha","Avistamientos"]]
df = df.set_index(['Lat_floor', 'Long_floor',"Fecha"])

#DataFrame total avistamientos de playas en esa cuadricula
df_sum = df.groupby(['Lat_floor', 'Long_floor',"Fecha"]).sum()

#DataFrame número de playas en esa cuadricula
df_count = df.groupby(['Lat_floor', 'Long_floor',"Fecha"]).count()

#Dataframe con el total de avistamientos y el número de playas
df_join = df_sum.join(df_count,lsuffix="I",rsuffix="R")
df_join.columns=['Suma', 'N_Playas']
df_join['Media_Playas'] = df_join.Suma /df_join.N_Playas

# DataFrame de avistamientos procesado
df_join.to_excel("2AvistamientosConRedondeoCoord-00833.xlsx")

In [7]:
df_join.head()

Suma  N_Playas  Media_Playas
Lat_floor  Long_floor Fecha                                   
-42.666667 -74.083333 2015-05-27     1         1           1.0
-41.833333 -73.833333 2014-06-04     1         1           1.0
           -73.500000 2015-05-23     1         1           1.0
-41.666667 -73.750000 2015-06-10    20         1          20.0
                      2015-06-11   151         1         151.0

## Exporto listado de coordenadas para utilizarlo en la web

In [8]:
df_web = df_join.reset_index().rename(columns={"Lat_floor": "Latitud", "Long_floor": "Longitud"})[['Latitud','Longitud']]
# df_web.to_excel("Playas.xlsx")

# Estructura Buena

In [9]:
# DF con playas
# df_playas = pd.read_excel('./2AvistamientosConRedondeoCoord-025.xlsx').reset_index()
df_playas = pd.read_excel('./2AvistamientosConRedondeoCoord-00833.xlsx').reset_index()
'''
Las celdas que estaban unidas por tener el mismo valor, tienen valores nulos
por lo que las relleno con us valor correspodiente
'''
df_playas = df_playas.fillna(method= 'ffill',axis=0)
# Cojo las columnas que me interesan
df_playas = df_playas[["Suma","Lat_floor","Long_floor","Fecha"]]
# Renombro las columnas 
df_playas = df_playas.rename(columns={"Suma": "Avistamientos", "Lat_floor": "Latitud", "Long_floor": "Longitud"})
# Ordeno por fecha
df_playas = df_playas.sort_values(by=["Fecha"])

In [35]:
df_playas # DataFrame con avistamientos en cada playa
cuadrantes = { # Numero de cuadrantes que se concatenarán con las playas
    'izquierda': 2 ,
    'arriba': 2 ,
    'abajo': 2
}
dias_desfase = 45
salto = 1/12 # Salto de coordenadas entre los cuadrantes
listado_archivos = os.listdir('../../descargas') # Listo todos los archivos de Copernicus
variables = ['Coord','Profundidad','mlotst','zos','bottomT','thetao','so','uo','vo']

In [11]:
def busca_archivo(fecha):
    texto ='_{}_'.format(str(fecha).split()[0].replace('-',''))
    archivo = [x for x in listado_archivos if str(texto) in x]
    data = xr.open_dataset('../../descargas/{}'.format(archivo[0])) # cargo el archivo
    return data # devuelvo dataset

def dame_datos(latitud,longitud,ds):
    return ds.sel({'latitude':latitud,'longitude': longitud})

def comprueba_datos(latitud,longitud,ds):
    valor = dame_datos(latitud,longitud,ds)
    while math.isnan(valor.mlotst[0]):
        longitud = longitud - salto
        valor = dame_datos(latitud,longitud,ds)
    return latitud,longitud # devuelvo las coordenadas con datos

def crea_DF():
    df_resultados = None
    df_resultados = pd.DataFrame(columns=["Avistamientos","Latitud","Longitud","Fecha",'Coord',"Profundidad","mlotst",'zos','bottomT','thetao','so','uo','vo']) # columnas básicas
    for izquierda in range(cuadrantes['izquierda']): # por cada cuadrante a la izquierda meto un grupo de columnas
        var_aux = [str(x+'_izq_'+str(izquierda+1)) for x in variables]
        df_aux = pd.DataFrame(columns=var_aux)
        df_resultados = df_resultados.join(df_aux)
        for superior in range (cuadrantes['arriba']): # por cada cuadrante a la superior meto un grupo de columnas
            var_aux = [x+'_sup_'+str(izquierda+1)+'.'+str(superior+1) for x in variables]
            df_aux = pd.DataFrame(columns=var_aux)
            df_resultados = df_resultados.join(df_aux)
        for inferior in range (cuadrantes['abajo']): # por cada cuadrante a la inferior meto un grupo de columnas    
            var_aux = [x+'_inf_'+str(izquierda+1)+'.'+str(inferior+1) for x in variables]
            df_aux = pd.DataFrame(columns=var_aux)
            df_resultados = df_resultados.join(df_aux)
    return df_resultados

def meto_en_df(df_resultados,row,coordenadas,df,indice,variables_actuales):
    for x in range(3):
        df_resultados.loc[indice + x,["Avistamientos","Latitud","Longitud","Fecha"]] = row.values.tolist() # meto en df_resulados los datos de ese cuadrante
        vars = df.loc[x,['depth', 'mlotst','zos','bottomT','thetao','so','uo','vo']].values.tolist()
        vars.insert(0,coordenadas)
        df_resultados.loc[indice + x,variables_actuales] = vars # meto coordenadas reales utilizadas y datos 
    return df_resultados

def dameCoordIzq(lat,long,n_saltos):
    return [lat,long-(salto*n_saltos)]

def dameCoordSuperior(lat,long,n_saltos):
    coordenadas = dameCoordIzq(lat,long,n_saltos)
    return [lat+(salto*n_saltos),long]

def dameCoordInferior(lat,long,n_saltos):
    return [lat-(salto*n_saltos),long]

def resta_fecha(fecha,dias):
    return fecha - pd.Timedelta(days=2)

In [12]:
def crea_estr():
    df_resultados = crea_DF() # creo df con columnas necesarias
    #print(df_resultados)
    total_lineas = df_playas.shape[0]
    pbar = tqdm(total = total_lineas)
    for contador,[index, row] in enumerate(df_playas.iterrows()): # por cada playa
#         print(contador,end='\r')
        pbar.update(1)
        ds = busca_archivo(resta_fecha(row['Fecha'],dias_desfase)) # busco el archivo de la fecha 
        coordenadas = comprueba_datos(row['Latitud'],row['Longitud'],ds) # miro a ver si el cuadrante tiene datos, sino cojo el siguiente por la izquierda

        df = dame_datos(coordenadas[0],coordenadas[1],ds).to_dataframe().reset_index() # dataframe con los datso oceanicos de esas coordenadas y en esa fecha
        df_resultados = meto_en_df(df_resultados,row,list(coordenadas),df,contador * 3,variables)
#         print('--{}'.format(contador))
        # datos de las celdas extras
        for izquierda in range(cuadrantes['izquierda']): # por cada cuadrante a la izquierda, añado datos
            coordenadas = dameCoordIzq(coordenadas[0],coordenadas[1],izquierda+1)
            df = dame_datos(coordenadas[0],coordenadas[1],ds).to_dataframe().reset_index() 
            var_aux = [x+'_izq_'+str(izquierda+1) for x in variables]
            df_resultados = meto_en_df(df_resultados,row,coordenadas,df,contador * 3,var_aux)
            coord_aux = coordenadas
            for superior in range (cuadrantes['arriba']): # por cada cuadrante a la superior, añado datos
                coord_aux = dameCoordSuperior(coord_aux[0],coord_aux[1],superior+1)
                df = dame_datos(coord_aux[0],coord_aux[1],ds).to_dataframe().reset_index() 
                var_aux = [x+'_sup_'+str(izquierda+1)+'.'+str(superior+1) for x in variables]
                df_resultados = meto_en_df(df_resultados,row,coord_aux,df,contador * 3,var_aux)
            coord_aux = coordenadas
            for inferior in range (cuadrantes['abajo']): # por cada cuadrante a la inferior, añado datos  
                coord_aux = dameCoordInferior(coord_aux[0],coord_aux[1],inferior+1)
                df = dame_datos(coord_aux[0],coord_aux[1],ds).to_dataframe().reset_index() 
                var_aux = [x+'_inf_'+str(izquierda+1)+'.'+str(inferior+1) for x in variables]
                df_resultados = meto_en_df(df_resultados,row,coord_aux,df,contador * 3,var_aux)

#         if contador == 5:
#             break
    return df_resultados

In [13]:
df_resultados = crea_estr()
# df_resultados.head()

In [14]:
df_resultados = df_resultados.reset_index().set_index(['Latitud','Longitud','Fecha','Avistamientos','Profundidad'])
df_resultados.to_excel('3EstructuraFinal.xlsx')
df_resultados.head()

index  \
Latitud    Longitud   Fecha      Avistamientos Profundidad          
-25.416667 -70.500000 2014-05-01 1             0.494025         0   
                                               5.078224         1   
                                               9.572997         2   
-18.500000 -70.333333 2014-05-01 1             0.494025         3   
                                               5.078224         4   

                                                                                               Coord  \
Latitud    Longitud   Fecha      Avistamientos Profundidad                                             
-25.416667 -70.500000 2014-05-01 1             0.494025     [-25.41666666666666, -70.58333333333333]   
                                               5.078224     [-25.41666666666666, -70.58333333333333]   
                                               9.572997     [-25.41666666666666, -70.58333333333333]   
-18.500000 -70.333333 2014-05-01 1             0.494025                  [-18.5, -70.33333333333333]   
                                               5.078224                  [-18.5, -70.33333333333333]   

                                                             mlotst       zos  \
Latitud    Longitud   Fecha      Avistamientos Profundidad                      
-25.416667 -70.500000 2014-05-01 1             0.494025     10.5289  0.166326   
                                               5.078224     10.5289  0.166326   
                                               9.572997     10.5289  0.166326   
-18.500000 -70.333333 2014-05-01 1             0.494025     10.5289   0.21485   
                                               5.078224     10.5289   0.21485   

                                                            bottomT   thetao  \
Latitud    Longitud   Fecha      Avistamientos Profundidad                     
-25.416667 -70.500000 2014-05-01 1             0.494025     12.3865  15.3455   
                                               5.078224     12.3865  15.2906   
                                               9.572997     12.3865  15.1727   
-18.500000 -70.333333 2014-05-01 1             0.494025     15.2723  17.7106   
                                               5.078224     15.2723  17.3605   

                                                                 so  \
Latitud    Longitud   Fecha      Avistamientos Profundidad            
-25.416667 -70.500000 2014-05-01 1             0.494025     34.7255   
                                               5.078224     34.7255   
                                               9.572997     34.7209   
-18.500000 -70.333333 2014-05-01 1             0.494025     34.8735   
                                               5.078224     34.8949   

                                                                   uo  \
Latitud    Longitud   Fecha      Avistamientos Profundidad              
-25.416667 -70.500000 2014-05-01 1             0.494025     -0.104373   
                                               5.078224     -0.111087   
                                               9.572997     -0.112308   
-18.500000 -70.333333 2014-05-01 1             0.494025    -0.0579852   
                                               5.078224    -0.0518815   

                                                                    vo  
Latitud    Longitud   Fecha      Avistamientos Profundidad              
-25.416667 -70.500000 2014-05-01 1             0.494025      0.0286874  
                                               5.078224    -0.00183111  
                                               9.572997      -0.028077  
-18.500000 -70.333333 2014-05-01 1             0.494025       0.126957  
                                               5.078224      0.0787378

In [36]:
for i in [0,7,14,30,45,60]:
    dias_desfase = i
    df_resultados = crea_estr()
    df_resultados = df_resultados.reset_index().set_index(['Latitud','Longitud','Fecha','Avistamientos','Profundidad'])
    df_resultados.to_excel('-EstructuraFinal{}dias.xlsx'.format(i))
# df_resultados.head()

## Estructura sin rellenar huecos sin datos

In [16]:
def crea_estr():
    df_resultados = crea_DF() # creo df con columnas necesarias
    #print(df_resultados)
    total_lineas = df_playas.shape[0]
    pbar = tqdm(total = total_lineas)
    for contador,[index, row] in enumerate(df_playas.iterrows()): # por cada playa
#         print(contador,end='\r')
        pbar.update(1)
        ds = busca_archivo(row['Fecha']) # busco el archivo de la fecha 
        coordenadas = [row['Latitud'],row['Longitud']]

        df = dame_datos(row['Latitud'],row['Longitud'],ds).to_dataframe().reset_index() # dataframe con los datso oceanicos de esas coordenadas y en esa fecha
        df_resultados = meto_en_df(df_resultados,row,[row['Latitud'],row['Longitud']],df,contador * 3,variables)
        
        # datos de las celdas extras
        for izquierda in range(cuadrantes['izquierda']): # por cada cuadrante a la izquierda, añado datos
            coordenadas = dameCoordIzq(coordenadas[0],coordenadas[1],izquierda+1)
            df = dame_datos(coordenadas[0],coordenadas[1],ds).to_dataframe().reset_index() 
            var_aux = [x+'_izq_'+str(izquierda+1) for x in variables]
            df_resultados = meto_en_df(df_resultados,row,coordenadas,df,contador * 3,var_aux)
            coord_aux = coordenadas
            for superior in range (cuadrantes['arriba']): # por cada cuadrante a la superior, añado datos
                coord_aux = dameCoordSuperior(coord_aux[0],coord_aux[1],superior+1)
                df = dame_datos(coord_aux[0],coord_aux[1],ds).to_dataframe().reset_index() 
                var_aux = [x+'_sup_'+str(izquierda+1)+'.'+str(superior+1) for x in variables]
                df_resultados = meto_en_df(df_resultados,row,coord_aux,df,contador * 3,var_aux)
            coord_aux = coordenadas
            for inferior in range (cuadrantes['abajo']): # por cada cuadrante a la inferior, añado datos  
                coord_aux = dameCoordInferior(coord_aux[0],coord_aux[1],inferior+1)
                df = dame_datos(coord_aux[0],coord_aux[1],ds).to_dataframe().reset_index() 
                var_aux = [x+'_inf_'+str(izquierda+1)+'.'+str(inferior+1) for x in variables]
                df_resultados = meto_en_df(df_resultados,row,coord_aux,df,contador * 3,var_aux)
        
    return df_resultados

In [17]:
df_resultados_con_vacios = crea_estr()

In [18]:
df_resultados_con_vacios.head()

,Avistamientos,Latitud,Longitud,Fecha,Coord,Profundidad,mlotst,zos,bottomT,thetao,so,uo,vo
0,1,-25.4167,-70.5,2014-05-01 00:00:00,"[-25.41666666666666, -70.5]",0.494025,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,-25.4167,-70.5,2014-05-01 00:00:00,"[-25.41666666666666, -70.5]",5.07822,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,-25.4167,-70.5,2014-05-01 00:00:00,"[-25.41666666666666, -70.5]",9.573,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,-18.5,-70.3333,2014-05-01 00:00:00,"[-18.5, -70.33333333333333]",0.494025,10.5289,0.219428,15.3323,17.6198,34.8888,-0.0714133,0.136113
4,1,-18.5,-70.3333,2014-05-01 00:00:00,"[-18.5, -70.33333333333333]",5.07822,10.5289,0.219428,15.3323,17.2711,34.9101,-0.0622578,0.0860622


In [19]:
df_resultados_con_vacios.to_excel('3EstructuraCoordEnTierra.xlsx')

# Rellenar datos de celdas vacios
Con datos de la coordenada más cercana por la izquierda

In [88]:
def comprueba_datos_thetao(latitud,longitud,ds):
    valor = dame_datos(latitud,longitud,ds)
    while math.isnan(valor.thetao[0][2]):
        longitud = float(longitud) - salto
        valor = dame_datos(latitud,longitud,ds)
    return latitud,longitud # devuelvo las coordenadas con datos

In [21]:
# datos_excel = pd.read_excel('EstructuraFinal.xlsx').reset_index() # leo estructura de datos que contiene huecos sin informacion
# datos = datos_excel.drop(['Unnamed: 0'],axis=1).sort_values(by=['Latitud','Longitud','Profundidad','Fecha']) 
datos_excel = df_resultados.copy().reset_index()
datos = datos_excel.sort_values(by=['Latitud','Longitud','Fecha']) 

In [22]:
datos

,Latitud,Longitud,Fecha,Avistamientos,Profundidad,index,Coord,mlotst,zos,bottomT,thetao,so,uo,vo
381,-42.666667,-74.083333,2015-05-27,1,0.494025,381,"[-42.66666666666666, -74.16666666666666]",49.5926,0.0946074,12.8193,12.8186,33.3384,0.0360118,0.0390637
382,-42.666667,-74.083333,2015-05-27,1,5.078224,382,"[-42.66666666666666, -74.16666666666666]",49.5926,0.0946074,12.8193,12.8171,33.3384,0.0152593,0.0317392
383,-42.666667,-74.083333,2015-05-27,1,9.572997,383,"[-42.66666666666666, -74.16666666666666]",49.5926,0.0946074,12.8193,12.8157,33.3384,0.00793481,0.0286874
21,-41.833333,-73.833333,2014-06-04,1,0.494025,21,"[-41.83333333333333, -73.83333333333333]",9.61333,0.0781274,11.4731,11.3251,33.4071,0.380261,0.0976592
22,-41.833333,-73.833333,2014-06-04,1,5.078224,22,"[-41.83333333333333, -73.83333333333333]",9.61333,0.0781274,11.4731,11.4438,33.5276,0.356456,0.0744652
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226,-18.500000,-70.333333,2014-08-17,1,5.078224,226,"[-18.5, -70.33333333333333]",10.5289,0.0256355,14.1831,15.1456,34.7652,-0.0140385,0.0531022
227,-18.500000,-70.333333,2014-08-17,1,9.572997,227,"[-18.5, -70.33333333333333]",10.5289,0.0256355,14.1831,14.8555,34.7652,-0.00793481,0.0250252
588,-18.416667,-70.333333,2015-08-18,1,0.494025,588,"[-18.41666666666666, -70.33333333333333]",14.9541,0.069277,15.7374,15.7447,34.8766,-0.108036,0.160527
589,-18.416667,-70.333333,2015-08-18,1,5.078224,589,"[-18.41666666666666, -70.33333333333333]",14.9541,0.069277,15.7374,15.7315,34.8766,-0.102542,0.140385


In [90]:
def rellenar_huecos(df):
    filas_nan = df[df.isna().any(axis=1)] # filas con valores vacios
    datos_copia = df.copy()
    total_lineas = filas_nan.shape[0]
    pbar = tqdm(total = total_lineas)
    for indice, fila in filas_nan.iterrows():
#         print(indice,fila)
        pbar.update(1)
        elem = fila.isna() # columnas con valores nan
        for index, value in elem.items():
            if value:
                nombre_original= index
                nombre = 'Coord'
                try:
                    nombre_original = index[:index.index('_')] # nombre de la columna en el DataSet
                    nombre = nombre + index[index.index('_'):] # nombre de la columna de coordenadas de ese punto
                except:
                    pass
                coord = fila[nombre]
                coordenadas = [0,0]
                coordenadas[0] = coord[1:coord.index(',')]
                coordenadas[1] = coord[coord.index(',')+1:coord.index(']')]
                archivo = busca_archivo(fila['Fecha'])
                coordenadas = comprueba_datos_thetao(coordenadas[0],coordenadas[1],archivo)
                datos_fila = dame_datos(coordenadas[0],coordenadas[1],archivo)
                valor =datos_fila.sel({'depth':fila['Profundidad']}).to_dataframe().reset_index()[nombre_original].values[0]
                datos_copia.loc[indice,index] = valor
    return datos_copia

In [91]:
datos_sin_huecos = rellenar_huecos(datos)

In [25]:
filas_nan_2 = datos_sin_huecos[datos_sin_huecos.isna().any(axis=1)] # compruebo que se hayan rellenado todos los vacios
filas_nan_2.head()

,Latitud,Longitud,Fecha,Avistamientos,Profundidad,index,Coord,mlotst,zos,bottomT,thetao,so,uo,vo


In [31]:
datos_sin_huecos = datos_sin_huecos.reset_index().set_index(['Latitud','Longitud','Fecha','Avistamientos','Profundidad']).drop(['index'],axis = 1)
datos_sin_huecos.to_excel('4EstructuraFinalSinMissings.xlsx')

In [32]:
datos_sin_huecos

Coord  \
Latitud    Longitud   Fecha      Avistamientos Profundidad                                             
-42.666667 -74.083333 2015-05-27 1.0           0.494025     [-42.66666666666666, -74.16666666666666]   
                                               5.078224     [-42.66666666666666, -74.16666666666666]   
                                               9.572997     [-42.66666666666666, -74.16666666666666]   
-41.833333 -73.833333 2014-06-04 1.0           0.494025     [-41.83333333333333, -73.83333333333333]   
                                               5.078224     [-41.83333333333333, -73.83333333333333]   
...                                                                                              ...   
-18.500000 -70.333333 2014-08-17 1.0           5.078224                  [-18.5, -70.33333333333333]   
                                               9.572997                  [-18.5, -70.33333333333333]   
-18.416667 -70.333333 2015-08-18 1.0           0.494025     [-18.41666666666666, -70.33333333333333]   
                                               5.078224     [-18.41666666666666, -70.33333333333333]   
                                               9.572997     [-18.41666666666666, -70.33333333333333]   

                                                               mlotst  \
Latitud    Longitud   Fecha      Avistamientos Profundidad              
-42.666667 -74.083333 2015-05-27 1.0           0.494025     49.592583   
                                               5.078224     49.592583   
                                               9.572997     49.592583   
-41.833333 -73.833333 2014-06-04 1.0           0.494025      9.613331   
                                               5.078224      9.613331   
...                                                               ...   
-18.500000 -70.333333 2014-08-17 1.0           5.078224     10.528886   
                                               9.572997     10.528886   
-18.416667 -70.333333 2015-08-18 1.0           0.494025     14.954070   
                                               5.078224     14.954070   
                                               9.572997     14.954070   

                                                                 zos  \
Latitud    Longitud   Fecha      Avistamientos Profundidad             
-42.666667 -74.083333 2015-05-27 1.0           0.494025     0.094607   
                                               5.078224     0.094607   
                                               9.572997     0.094607   
-41.833333 -73.833333 2014-06-04 1.0           0.494025     0.078127   
                                               5.078224     0.078127   
...                                                              ...   
-18.500000 -70.333333 2014-08-17 1.0           5.078224     0.025636   
                                               9.572997     0.025636   
-18.416667 -70.333333 2015-08-18 1.0           0.494025     0.069277   
                                               5.078224     0.069277   
                                               9.572997     0.069277   

                                                              bottomT  \
Latitud    Longitud   Fecha      Avistamientos Profundidad              
-42.666667 -74.083333 2015-05-27 1.0           0.494025     12.819330   
                                               5.078224     12.819330   
                                               9.572997     12.819330   
-41.833333 -73.833333 2014-06-04 1.0           0.494025     11.473098   
                                               5.078224     11.473098   
...                                                               ...   
-18.500000 -70.333333 2014-08-17 1.0           5.078224     14.183142   
                                               9.572997     14.183142   
-18.416667 -70.333333 2015-08-18 1.0           0.494025     15.737389   
                                               5.078

In [92]:
for i in [0,7,14,30,45,60]:
    df = pd.read_excel('-EstructuraFinal{}dias.xlsx'.format(i)).reset_index().drop(['level_0','index'],axis = 1).fillna(method='ffill', axis=0)
    datos = df.sort_values(by=['Latitud','Longitud','Fecha']) 
    datos_sin_huecos = rellenar_huecos(datos)
    datos_sin_huecos = datos_sin_huecos.reset_index().set_index(['Latitud','Longitud','Fecha','Avistamientos','Profundidad'])
    datos_sin_huecos.to_excel('-EstructuraFinal{}dias.xlsx'.format(i))
# df_resultados.head()